In [1]:
# Force matplotlib to not use any Xwindows backend.
import matplotlib
matplotlib.use('Agg')
import viz
from deepsurv import utils
import deepsurv

import argparse
from collections import defaultdict
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import h5py
import uuid
from sklearn.utils import resample 
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# rpy2 lets you run R within Python
import rpy2 
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
# Run a RandomSurvivalForest
rfSRC = importr('randomForestSRC')

# Converts pandas Dataframes into R Dataframes
from rpy2.robjects import pandas2ri
pandas2ri.activate()

import logging

import time
localtime   = time.localtime()
TIMESTRING  = time.strftime("%m%d%Y%M", localtime)

DURATION_COL = 'time'
EVENT_COL = 'status'

In [2]:
import h5py
filename = 'C:/Users/ASUS/Dropbox/석사학위논문/DeepSurv-master/data/whas_train_test.h5'
f = h5py.File(filename, 'r')

#Get the HDF5 group
group = f['train']

#Checkout what keys are inside that group.
for key in group.keys():
    print(key)

data_x = group['x'].value
data_t = group['t'].value
data_e = group['e'].value

#Do whatever you want with data

#After you are done
f.close()

train_df = pd.DataFrame(data_x)
train_df['Time'] = np.int32(data_t)
train_df['Event'] = data_e

e
t
x


C:\Users\ASUS\Anaconda3\lib\site-packages\h5py\_hl\dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [3]:
train_df, val_df = train_test_split(train_df, test_size = 0.2)

In [2]:
# rpy2 lets you run R within Python
import rpy2 
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
# Run a RandomSurvivalForest
rfSRC = importr('randomForestSRC')

# Converts pandas Dataframes into R Dataframes
from rpy2.robjects import pandas2ri
pandas2ri.activate()

import logging


def evaluate_model(model, dataset, bootstrap = False, trt_idx = None):
    def ci(model):
        def rsf_ci(**kwargs):
            data = utils.format_dataset_to_df(kwargs, 't', 'e', trt_idx)
            pred_test = rfSRC.predict_rfsrc(model, data)
            return 1 - pred_test.rx('err.rate')[0][-1]
        return rsf_ci

    metrics = {}

    # Calculate c_index
    metrics['c_index'] = ci(model)(**dataset)
    if bootstrap:
        metrics['c_index_bootstrap'] = utils.bootstrap_metric(ci(model), dataset)
    
    return metrics

# WHAS

In [19]:
if __name__ == '__main__':

    datasets = utils.load_datasets('C:/Users/ASUS/Dropbox/석사학위논문/DeepSurv-master/data/whas_train_test.h5')

    # Train CPH model
    print("Training RSF Model")
    train_df = utils.format_dataset_to_df(datasets['train'], DURATION_COL, EVENT_COL)
    surv_f = robjects.r("as.formula(Surv(time,status) ~ .)")
    rfsc = rfSRC.rfsrc(surv_f, train_df, ntree=100)

    print('Train/Valid C-Index:', 1 - rfsc.rx('err.rate')[0][-1])
    if 'valid' in datasets:
        metrics = evaluate_model(rfsc, datasets['valid'])
        print("Valid metrics: " + str(metrics))

    if 'test' in datasets:
        metrics = evaluate_model(rfsc, datasets['test'], bootstrap=True)
        print("Test metrics: " + str(metrics))

Training RSF Model
Train/Valid C-Index: 0.832593473741147
Test metrics: {'c_index': 0.8348423142943691, 'c_index_bootstrap': {'mean': 0.8372622309877197, 'confidence_interval': (0.8339451292911861, 0.8405793326842532)}}


# METABRIC

In [7]:
if __name__ == '__main__':

    datasets = utils.load_datasets('C:/Users/ASUS/Dropbox/석사학위논문/DeepSurv-master/data/metabric_IHC4_clinical_train_test.h5')

    # Train CPH model
    print("Training RSF Model")
    train_df = utils.format_dataset_to_df(datasets['train'], DURATION_COL, EVENT_COL)
    surv_f = robjects.r("as.formula(Surv(time,status) ~ .)")
    rfsc = rfSRC.rfsrc(surv_f, train_df, ntree=100)

    print('Train/Valid C-Index:', 1 - rfsc.rx('err.rate')[0][-1])
    if 'valid' in datasets:
        metrics = evaluate_model(rfsc, datasets['valid'])
        print("Valid metrics: " + str(metrics))

    if 'test' in datasets:
        metrics = evaluate_model(rfsc, datasets['test'], bootstrap=True)
        print("Test metrics: " + str(metrics))

Training RSF Model
Train/Valid C-Index: 0.6531827893703416
Test metrics: {'c_index': 0.6387032707863541, 'c_index_bootstrap': {'mean': 0.6401059391159007, 'confidence_interval': (0.6363905541443751, 0.6438213240874264)}}


# GBSG

In [8]:
if __name__ == '__main__':

    datasets = utils.load_datasets('C:/Users/ASUS/Dropbox/석사학위논문/DeepSurv-master/data/gbsg_cancer_train_test.h5')

    # Train CPH model
    print("Training RSF Model")
    train_df = utils.format_dataset_to_df(datasets['train'], DURATION_COL, EVENT_COL)
    surv_f = robjects.r("as.formula(Surv(time,status) ~ .)")
    rfsc = rfSRC.rfsrc(surv_f, train_df, ntree=100)

    print('Train/Valid C-Index:', 1 - rfsc.rx('err.rate')[0][-1])
    if 'valid' in datasets:
        metrics = evaluate_model(rfsc, datasets['valid'])
        print("Valid metrics: " + str(metrics))

    if 'test' in datasets:
        metrics = evaluate_model(rfsc, datasets['test'], bootstrap=True)
        print("Test metrics: " + str(metrics))

Training RSF Model
Train/Valid C-Index: 0.660127137003674
Test metrics: {'c_index': 0.6677372580839043, 'c_index_bootstrap': {'mean': 0.6685059341777223, 'confidence_interval': (0.6653539405911867, 0.6716579277642579)}}


# SUPPORT

In [ ]:
if __name__ == '__main__':

    datasets = utils.load_datasets('C:/Users/ASUS/Dropbox/석사학위논문/DeepSurv-master/data/support_train_test.h5')

    # Train CPH model
    print("Training RSF Model")
    train_df = utils.format_dataset_to_df(datasets['train'], DURATION_COL, EVENT_COL)
    surv_f = robjects.r("as.formula(Surv(time,status) ~ .)")
    rfsc = rfSRC.rfsrc(surv_f, train_df, ntree=100)

    print('Train/Valid C-Index:', 1 - rfsc.rx('err.rate')[0][-1])
    if 'valid' in datasets:
        metrics = evaluate_model(rfsc, datasets['valid'])
        print("Valid metrics: " + str(metrics))

    if 'test' in datasets:
        metrics = evaluate_model(rfsc, datasets['test'], bootstrap=True)
        print("Test metrics: " + str(metrics))

# NWTCO

In [6]:
#nwtco 데이터 불러오기
nwtco = pd.read_csv('C:/Users/ASUS/Dropbox/석사학위논문/DeepSurv-master/data/nwtco.csv')
train_df = nwtco.drop('seqno', axis = 1)
train_df = train_df.rename(columns = {'rel':'Event', 'edrel':'Time'})

#censored number
c_number = train_df['Event'].value_counts()[0]
feat_num = len(list(train_df.columns))
total_number = len(train_df['Event'])
c_rate = (c_number / total_number) * 100

print("total_number : ", total_number)
print("feature_number : ", feat_num)
print("censored_number : ", c_number)
print("censored_rate : " , c_rate)

#data split
train_df, test_df = train_test_split(train_df, test_size = 0.2)
train_df, val_df = train_test_split(train_df, test_size = 0.2)

total_number :  4028
feature_number :  8
censored_number :  3457
censored_rate :  85.82423038728898


In [9]:
surv_f = robjects.r("as.formula(Surv(Time,Event) ~ .)")
rfsc = rfSRC.rfsrc(surv_f, train_df, ntree=50)
pred_test = rfSRC.predict_rfsrc(rfsc, test_df)
rsf_ci = 1 - pred_test.rx('err.rate')[0][-1]
rsf_ci

0.7273909351448891

In [19]:
n_iterations = 1000
n_size = int(len(train_df) * 0.50)

# run bootstrap
stats = list()
for i in range(n_iterations):
    train = resample(train_df, n_samples=n_size)
    surv_f = robjects.r("as.formula(Surv(Time,Event) ~ .)")
    rfsc = rfSRC.rfsrc(surv_f, train, ntree=50)
    pred_test = rfSRC.predict_rfsrc(rfsc, test_df)
    score = 1 - pred_test.rx('err.rate')[0][-1]
    stats.append(score)
    
# confidence intervals
alpha = 0.95
p = ((1.0-alpha)/2.0) * 100
lower = max(0.0, np.percentile(stats, p))
p = (alpha+((1.0-alpha)/2.0)) * 100
upper = min(1.0, np.percentile(stats, p))
print('%.1f confidence interval %.1f%% and %.1f%%' % (alpha*100, lower*100, upper*100))

95.0 confidence interval 84.3% and 88.0%


In [22]:
np.mean(stats)

0.8619736234329921

# Non-linear

In [4]:
#generate data
import deepsurv

def generate_data(treatment_group = False):
    np.random.seed(123)
    sd = deepsurv.datasets.SimulatedData(5, num_features = 15,
        treatment_group = treatment_group)
    train_data = sd.generate_data(1000,method = 'gaussian')
    valid_data = sd.generate_data(400,method = 'gaussian')
    test_data = sd.generate_data(400,method = 'gaussian')

    return train_data, valid_data, test_data

train_data, valid_data, test_data = generate_data()

In [5]:
if __name__ == '__main__':

    # Train CPH model
    print("Training RSF Model")
    train_df = utils.format_dataset_to_df(train_data, 't', 'e')
    surv_f = robjects.r("as.formula(Surv(t,e) ~ .)")
    rfsc = rfSRC.rfsrc(surv_f, train_df, ntree=3)

    print('Train/Valid C-Index:', 1 - rfsc.rx('err.rate')[0][-1])
    metrics = evaluate_model(rfsc, valid_data)
    print("Valid metrics: " + str(metrics))

    metrics = evaluate_model(rfsc, test_data, bootstrap=True)
    print("Test metrics: " + str(metrics))

Training RSF Model
Train/Valid C-Index: 0.5598493770081854
Valid metrics: {'c_index': 0.5786100628930818}
Test metrics: {'c_index': 0.5718484898289442, 'c_index_bootstrap': {'mean': 0.5732000893330033, 'confidence_interval': (0.570152048617065, 0.5762481300489415)}}


# Linear

In [10]:
#generate data
import deepsurv

def generate_data(treatment_group = False):
    np.random.seed(123)
    sd = deepsurv.datasets.SimulatedData(5, num_features = 15,
        treatment_group = treatment_group)
    train_data = sd.generate_data(1000,method = 'linear')
    valid_data = sd.generate_data(400,method = 'linear')
    test_data = sd.generate_data(400,method = 'linear')

    return train_data, valid_data, test_data

train_data, valid_data, test_data = generate_data()

In [11]:
if __name__ == '__main__':

    # Train CPH model
    print("Training RSF Model")
    train_df = utils.format_dataset_to_df(train_data, 't', 'e')
    surv_f = robjects.r("as.formula(Surv(t,e) ~ .)")
    rfsc = rfSRC.rfsrc(surv_f, train_df, ntree=100)

    print('Train/Valid C-Index:', 1 - rfsc.rx('err.rate')[0][-1])
    metrics = evaluate_model(rfsc, valid_data)
    print("Valid metrics: " + str(metrics))

    metrics = evaluate_model(rfsc, test_data, bootstrap=True)
    print("Test metrics: " + str(metrics))

Training RSF Model
Train/Valid C-Index: 0.775297067435245
Valid metrics: {'c_index': 0.747487453310914}
Test metrics: {'c_index': 0.7593129571577848, 'c_index_bootstrap': {'mean': 0.7602748345358316, 'confidence_interval': (0.7579763993989844, 0.7625732696726787)}}
